In [1]:
# !pip install PyPDF2
# !pip install pytesseract
# !pip install pdf2image
# !apt-get install -y poppler-utils
# !apt install tesseract-ocr
# !pip install selenium

In [2]:
import os
import re
import glob
import time
import PyPDF2
import pandas as pd
from bs4 import BeautifulSoup
from collections import defaultdict
from selenium import webdriver
from pdf2image import convert_from_path
import pytesseract


In [3]:
#HARD CODE STIMULATION PAGE NUMBER FOR EACH FILE
stimulation_page = {'W21796.pdf':19,'W22099.pdf':5,'W22220.pdf':15,'W22221.pdf':4,'W21266.pdf':19,'W20864.pdf':16,'W20863.pdf':15,'W20407.pdf':18,
                    'W20197.pdf':84,'W22247.pdf':47,'W22249.pdf':7,'W22731.pdf':8,'W22740.pdf':15,'W23230.pdf':5,'W23359.pdf':6,'W23360.pdf':5,
                    'W23361.pdf':6,'W23362.pdf':5,'W23363.pdf':6,'W23364.pdf':5,'W23365.pdf':6,'W23366.pdf':5,'W23367.pdf':9,'W23368.pdf':6,
                    'W23369.pdf':8,'W23370.pdf':5,'W23371.pdf':7,'W23372.pdf':6,'W25156.pdf':15,'W25157.pdf':9,'W25158.pdf':11,'W25159.pdf':8,
                    'W25160.pdf':9,'W25571.pdf':4,'W28190.pdf':4,'W28194.pdf':4,'W28303.pdf':6,'W28342.pdf':4,'W28394.pdf':6,'W28425.pdf':13,
                    'W28554.pdf':4,'W28557.pdf':11,'W28599.pdf':6,'W28600.pdf':7,'W28601.pdf':7,'W28633.pdf':7,'W28634.pdf':5,'W28636.pdf':5,
                    'W28648.pdf':6,'W28649.pdf':6,'W28651.pdf':5,'W28652.pdf':77,'W28654.pdf':8,'W28655.pdf':5,'W28658.pdf':3,'W28744.pdf':11,
                    'W28754.pdf':6,'W28755.pdf':6,'W28756.pdf':5,'W28976.pdf':6,'W28978.pdf':3,'W29242.pdf':5,'W29244.pdf':5,'W29316.pdf':5,
                    'W29317.pdf':7,'W29334.pdf':6,'W30188.pdf':22,'W30189.pdf':4,'W30789.pdf':5,'W36047.pdf':44,'W90244.pdf':11,'W90258.pdf':13,
                    'W90329.pdf':7}

In [4]:
# 2. Connect to Drive and Install Libraries
def connect_drive():
    from google.colab import drive
    drive.mount('/content/drive')

In [5]:
# 3. Extract Information from PDF Files
# This function determines the pattern to extract the well name based on the given PDF filename.
def well_name_func(pdf):
    if pdf.split('/')[-1] == 'W23359.pdf':
        well_name_pattern = 'Well Name .+ \n(.*?)(?:\n|$)'
    elif pdf.split('/')[-1] == 'W28601.pdf':
        well_name_pattern = 'Well Name .+ \n(.*?)(?:\n|$|Before After)'
    elif pdf.split('/')[-1] in ['W20863.pdf','W22731.pdf','W20864.pdf','W20407.pdf']:
        well_name_pattern = 'Well or Facility Name : (.*?)(?:\n|$)'
    else:
        well_name_pattern = 'Well Name and Number \n(.*?)(?:\n|$)'
    return well_name_pattern

In [6]:
# This function determines the pattern to extract the API number based on the given PDF filename.
def api_func(pdf):
    if pdf.split('/')[-1] == 'W90258.pdf':
        API_pattern = '\d{2}\s?-\s?\d{3}\s?-\s?\d{5}'
    elif pdf.split('/')[-1] == 'W20407.pdf':
        API_pattern = '31-19H.*API\s(.*)'
    elif pdf.split('/')[-1] in ['W21796.pdf','W20863.pdf','W21266.pdf']:
        API_pattern = '\d{3}\s?-?\s?\d{5}'
    else:
        API_pattern = '\d{2}-\d{3}-\d{5}'
    return API_pattern

In [7]:
# This function fixes the API number format based on the given PDF filename and the matched API pattern.
def api_fix_func(pdf, API_pattern_match):
    if pdf.split('/')[-1] in ['W21796.pdf','W20863.pdf','W21266.pdf']:
        api_num = API_pattern_match.group(0)
        api_num_fixed = '33-'+api_num.split()[0]+'-'+api_num.split()[1]
    elif pdf.split('/')[-1] == 'W20407.pdf':
        api_num = API_pattern_match.group(1).strip()
        api_num_fixed = api_num[:2]+'-'+api_num[3:6]+'-'+api_num[6:]
    elif pdf.split('/')[-1] == 'W90258.pdf':
        api_num = API_pattern_match.group(0)
        api_num_fixed = api_num.replace(' ',"")
    else:
        api_num_fixed = API_pattern_match.group(0)
    return api_num_fixed

In [8]:
def extract_well_data_from_pdfs(folder_path):
    # Get a list of all PDF files in the given folder
    pdf_files = glob.glob(os.path.join(folder_path, "*.pdf"))

    # Create a dictionary to store extracted data from each PDF
    files_dict = defaultdict(dict)

    # Iterate over each PDF file
    for pdf in pdf_files:
        # Open the PDF file for reading
        pdf_file = open(pdf, "rb")
        # Initialize a PDF reader object
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        # Get the number of pages in the PDF
        page_numbers = len(pdf_reader.pages)

        # Iterate over each page in the PDF
        for page_num in range(page_numbers):
            try:
                # Get the text content of the page
                page = pdf_reader.pages[page_num]
                page_text = page.extract_text()

                # Get patterns to match well name and API number
                well_name_pattern = well_name_func(pdf)
                API_pattern = api_func(pdf)

                # Try matching the patterns against the page text
                well_name_match = re.search(well_name_pattern, page_text)
                API_pattern_match = re.search(API_pattern, page_text)

                # If a match for well name is found, store it in the dictionary
                if well_name_match:
                    files_dict[pdf.split('/')[-1]]['Well_name'] = well_name_match.group(1).strip()
                    # If both well name and API number are found for a PDF, break out of the loop
                    if len(list(files_dict[pdf.split('/')[-1]].keys())) == 2:
                        break

                # If a match for API number is found, process it further and store in the dictionary
                if API_pattern_match:
                    api_num_fixed = api_fix_func(pdf, API_pattern_match)
                    files_dict[pdf.split('/')[-1]]['API#'] = api_num_fixed.strip()
                    # If both well name and API number are found for a PDF, break out of the loop
                    if len(list(files_dict[pdf.split('/')[-1]].keys())) == 2:
                        print("Extract well data from pdfs:", files_dict[pdf.split('/')[-1]])
                        break
            except:
                # If there's any error in processing a page, move on to the next page
                continue

            # If we've reached the last page of the PDF and haven't found both pieces of data, print the PDF name
            if page_num == page_numbers-1:
                print(pdf)



    # Return the dictionary with extracted data from all PDFs
    return files_dict

In [9]:
# Extracts text from a specific page of a given PDF using OCR.
def extract_text_from_pdf(pdf_path, page_number):
    # Convert the specified PDF page to an image at 500 DPI
    pages = convert_from_path(pdf_path, 500, first_page=page_number, last_page=page_number)
    # Use pytesseract to extract text from the image
    text = pytesseract.image_to_string(pages[0])
    return text

In [10]:
# Extracts specific data from a given string based on patterns and positions.
def extract_row1_data(row1_string):

    # Remove any "|" characters from the string
    row1_string = row1_string.replace("|", "")

    # Split the string into a list of words
    row1_list = row1_string.split()

    # Initialize default values for extracted data
    date_stim, stim_form, top, bottom, stim_stages, volume, vol_units = [None]*7

    # Extract data based on conditions and positions
    if len(row1_list) >= 7:
        if re.match("^[a-zA-Z]+$", row1_list[0]) is None:
            date_stim = row1_list[0]
        stim_form = row1_list[1]
        if (stim_form == 'Three') or (stim_form == '3'):
            stim_form = 'Three Forks'
        if row1_list[2].isdigit():
            top = row1_list[-5]
        if row1_list[3].isdigit():
            bottom = row1_list[-4]
        if row1_list[4].isdigit():
            stim_stages = row1_list[-3]
        if row1_list[-2].isdigit():
            volume = row1_list[-2]
        if row1_list[-1] == 'Barrels':
            vol_units = row1_list[-1]

    return date_stim, stim_form, top, bottom, stim_stages, volume, vol_units

In [11]:
# Extracts type of treatment from the given string based on specific patterns.
def extract_type_treat_data(matched_string):
    type_treat = None
    if matched_string == 'Sand Frac':
        type_treat = matched_string
    elif 'Sand Frac' in matched_string:
        type_treat = 'Sand Frac'
    return type_treat

In [12]:
# Extracts data from a given row2_string based on specific patterns and positions.
def extract_row2_data(row2_string):

    # Remove any "|" characters from the string
    row2_string = row2_string.replace("|", "")

    # Split the string into a list of words
    row2_list = row2_string.split()

    # Initialize default values for extracted data
    acid, proppant, pressure, rate = [None]*4

    # Extract data based on conditions and positions
    if len(row2_list) >= 3:
        if len(row2_list) == 4:
            if row2_list[0].isdigit() and len(row2_list[0]) <= 3:
                acid = row2_list[0]
            elif row2_list[0].isdigit() and len(row2_list[0]) > 3:
                proppant = row2_list[0]
            else:
                if row2_list[-3].isdigit():
                    proppant = row2_list[-3]
                pressure = row2_list[-2]
                rate = row2_list[-1]
        else:
            proppant = row2_list[0]
            pressure = row2_list[1]
            rate = row2_list[2]

    return acid, proppant, pressure, rate

In [13]:
# Extracts details data based on specific conditions.
def extract_details_data(details):
    # Check specific conditions and return None if any of them are met
    if details.startswith('\nDate') or len(details) >= 500:
        return None
    return details

In [14]:
# Extracts stimulation data from a list of PDF files.
def extract_stimulation_data_from_pdfs(pdf_files):

    stimulation_dict = defaultdict(dict)

    # Iterate over each PDF file
    for pdf in pdf_files:
        pdf_file = open(pdf, "rb")
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        # Retrieve the page number for extraction from a predefined dictionary
        page_number = stimulation_page.get(pdf.split('/')[-1], None)

        date_stim, stim_form, top, bottom, stim_stages, volume, vol_units, type_treat, acid, proppant, pressure, rate, details= [None]*13

        # If a page number is found, extract data from it
        if page_number:
            text = extract_text_from_pdf(pdf, page_number)

            row1_sim_pattern = 'Units\n(.*)'
            type_treat_pattern = 'Min\)\n(\D*)'
            row2_sim_pattern = 'Min\)\n(.*)'
            details_sim_pattern = re.compile(r"Details\n(.*?).\n\nDa", re.DOTALL)

            row1_sim_pattern_match = re.search(row1_sim_pattern, text)
            type_treat_pattern_match = re.search(type_treat_pattern, text)
            row2_sim_pattern_match = re.search(row2_sim_pattern, text)
            details_sim_pattern_match = re.search(details_sim_pattern, text)

            if row1_sim_pattern_match:
                row1_string = row1_sim_pattern_match.group(1)
                # Extract row1 data from the text
                date_stim, stim_form, top, bottom, stim_stages, volume, vol_units = extract_row1_data(row1_string)

            if type_treat_pattern_match:
                matched_string = type_treat_pattern_match.group(1)
                # Extract type of treatment from the text
                type_treat = extract_type_treat_data(matched_string)

            if row2_sim_pattern_match:
                if type_treat is not None:
                    row2_string = row2_sim_pattern_match.group(1).replace(type_treat,"")
                # Extract row2 data from the text
                acid, proppant, pressure, rate = extract_row2_data(row2_string)

            if details_sim_pattern_match:
                # Extract additional details from the text
                details = details_sim_pattern_match.group(1)
                details = extract_details_data(details)

            # Populate the dictionary with extracted data
            stimulation_dict[pdf.split('/')[-1]] = {
                'Date_stimulated': date_stim,
                'Stim_formation': stim_form,
                'Top': top,
                'Bottom': bottom,
                'Stim_stages': stim_stages,
                'Volume': volume,
                'Units': vol_units,
                'Type_treatment': type_treat,
                'Acid%': acid,
                'Proppant_Lbs': proppant,
                'Max_Pressure': pressure,
                'Max_Rate': rate,
                'Details': details
            }

            print("extract stimulation data from pdfs:", stimulation_dict[pdf.split('/')[-1]])

        pdf_file.close()

    return stimulation_dict

In [15]:
# 4. Web Scraping Information
def scrape_well_data(api_df):

    # api_df is the dataframe

    driver = webdriver.Chrome()
    url = 'https://www.drillingedge.com/search'
    driver.get(url)
    time.sleep(10)
    api_input = driver.find_element_by_name('33-053-06028')

    web_df = pd.DataFrame()

    # for api_number in api_numbers:
        # ... [Rest of the code for web scraping]
    driver.quit()

    return web_df

In [27]:
# Merge the data from two dictionaries into a single dataframe and save to a CSV.

def merge_and_save_dataframes(files_dict, stimulation_dict, output_folder):
    # Convert the dictionaries to dataframes
    df = pd.DataFrame.from_dict(files_dict,orient='index')
    df_stimulation = pd.DataFrame.from_dict(stimulation_dict,orient='index')

    # Merge the two dataframes on their indexes (horizontally)
    merged_df = pd.concat([df, df_stimulation], axis=1)
    print("Information Extracted from PDF: ", merged_df)

    # Save the merged dataframe to a CSV file
    merged_df.to_csv(output_folder + "Task_PDF_original.csv")

    return merged_df

In [28]:
def preprocess_and_save_dataframes(df, output_folder, task):

    if task == "a":
        # Remove HTML tags
        def remove_html_tags(text):
            clean = re.compile('<.*?>')
            return re.sub(clean, '', str(text))

        # Remove special characters (except for alphanumeric characters and spaces)
        def remove_special_characters(text):
            return re.sub('[^A-Za-z0-9\s\./-]+', '', str(text))

        processed_df = df.applymap(remove_html_tags)
        processed_df = processed_df.applymap(remove_special_characters)

        # Handle missing data
        # Replace NaN values in numeric columns with 0 and in string columns with "N/A"
        for column in processed_df.columns:
            if processed_df[column].dtype == 'object':  # If column is of object type, it's considered a string
                processed_df[column].fillna("N/A", inplace=True)
            else:  # If column is numeric
                processed_df[column].fillna(0, inplace=True)

        processed_df.replace("None", "nan", inplace=True)

        # Preprocess the "\n"
        processed_df.replace("\n", ";", regex=True, inplace=True)
        processed_df['Details'] = processed_df['Details'].str.replace(r"(;[\s;]+)", ";", regex=True).str.strip(";")

        # Save the merged dataframe to a CSV file
        processed_df.to_csv(output_folder+'Task1_PDF_preprocessed.csv', index=False)

    else:
        # Save the merged dataframe to a CSV file
        processed_df.to_csv(output_folder+'Task1_Web_preprocessed.csv', index=False)

        pass


    return processed_df

In [18]:
def store_db(df):

  # TODO: Store A to database

  pass

In [19]:
def update_db(df):

  # TODO: Update B to database

  pass

In [20]:
# Running the code
if __name__ == "__main__":
    # Establish a connection to Google Drive (assuming a relevant function is defined elsewhere)
    connect_drive()

    # Define the folder path containing the PDF files and retrieve the list of PDF files
    folder_path = '/content/drive/Shareddrives/560_GROUP/DSCI560_Lab5'
    pdf_files = glob.glob(os.path.join(folder_path, "*.pdf"))

    # Extract well data and stimulation data from the PDFs
    files_dict = extract_well_data_from_pdfs(folder_path)
    stimulation_dict = extract_stimulation_data_from_pdfs(pdf_files)

    # Define the output folder path
    output_folder = '/content/drive/Shareddrives/560_GROUP/Lab5_outputs/'

    # Merge the extracted data and save to a CSV
    merged_df = merge_and_save_dataframes(files_dict, stimulation_dict, output_folder)

    # preprocess the original data
    processed_df = preprocess_and_save_dataframes(merged_df, output_folder, "a")


    # store_db(processed_df)

    # merge_df is the unprep dataframe that from a,
    api_numbers = merged_df["API#"]
    web_df = scrape_well_data(api_numbers)

    # processed_web_df = preprocess_and_save_dataframes(web_df, output_folder, "b")

    # update_db(processed_web_df)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Shareddrives/560_GROUP/DSCI560_Lab5/W11920.pdf
Extract well data from pdfs: {'Well_name': 'MAGNUM  3-36-25H', 'API#': '33-053-04069'}
Extract well data from pdfs: {'Well_name': 'MAGNUM 2-36-25H', 'API#': '33-053-03944'}
Extract well data from pdfs: {'Well_name': 'Colville 5301 44-12T', 'API#': '33-053-04981'}
Extract well data from pdfs: {'Well_name': 'DAHL FEDERAL 2-15H', 'API#': '33-002-58854'}
Extract well data from pdfs: {'Well_name': 'Magnum 1-36-25H', 'API#': '33-053-03943'}
Extract well data from pdfs: {'Well_name': '(see details', 'API#': '33-053-06010'}
Extract well data from pdfs: {'Well_name': 'DAHL 15-11H', 'API#': '33-002-58854'}
Extract well data from pdfs: {'Well_name': 'BRAY  5301 43-12H', 'API#': '33-053-03911'}
Extract well data from pdfs: {'Well_name': 'FOLEY FEDERAL  5301 43-12H', 'API#': '33-000-10000'}
Extract well data fr

/usr/local/lib/python3.10/dist-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /90ms-RKSJ-H not implemented yet
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PyPDF2/_cmap.py:142: PdfReadWarning: Advanced encoding /90ms-RKSJ-V not implemented yet
  warnings.warn(


Extract well data from pdfs: {'Well_name': 'Atlanta 13-6H', 'API#': '33-105-02720'}
Extract well data from pdfs: {'Well_name': 'AUmta 12-6H Sec 5, 6, 7, & 8 T153N R101W', 'API#': '33-105-02721'}
Extract well data from pdfs: {'Well_name': 'Atlanta Federal 7-6H', 'API#': '33-105-02726'}
Extract well data from pdfs: {'Well_name': 'Atlanta Federal 9-6H', 'API#': '33-105-02724'}
Extract well data from pdfs: {'Well_name': 'Tallahassee 3-16H', 'API#': '33-053-04853'}
Extract well data from pdfs: {'Well_name': 'BUCK SHOT SWD 5300 31-31', 'API#': '33-053-90244'}
Extract well data from pdfs: {'Well_name': 'Atlanta Federal 5-6H', 'API#': '33-105-02728'}
Extract well data from pdfs: {'Well_name': 'Atlanta 1-6H', 'API#': '33-105-02732'}
Extract well data from pdfs: {'Well_name': 'Wade Federal 5300 21-30 12T', 'API#': '33-053-06129'}
Extract well data from pdfs: {'Well_name': 'Wade Federal 5300 41-30 4T -+ Se~ t>c.~o...~ls', 'API#': '33-053-06051'}
Extract well data from pdfs: {'Well_name': 'Kline F

In [26]:
    # processed_df is the preprocessd dataframe from task a. You can check this format.
    api_df = processed_df[["Well_name", "API#"]]

    api_df
    # # test your function here
    # web_df = scrape_well_data(api_df)
    # web_df

,Well_name,API#
W11920.pdf,CORPS OF ENGINEERS 31-10,nan
W22731.pdf,MAGNUM 3-36-25H,33-053-04069
W22249.pdf,MAGNUM 2-36-25H,33-053-03944
W25571.pdf,Colville 5301 44-12T,33-053-04981
W21796.pdf,DAHL FEDERAL 2-15H,33-002-58854
...,...,...
W30789.pdf,KLINE FEDERAL 5300 31-18 15T,33-053-06755
W23362.pdf,Atlanta 11-6H,33-105-02722
W28754.pdf,Kline Federal 5300 31-18 8B,33-053-00605
W23359.pdf,Atlanta 14-SH,33-105-02719
